In [ ]:
from google.colab import files
uploaded = files.upload()  # Select the dataset CSV


Saving Train.csv to Train.csv
Saving Mapping.csv to Mapping.csv
Saving OutputFormat.csv to OutputFormat.csv
Saving Test.csv to Test.csv


In [ ]:
from google.colab import files
uploaded = files.upload()  # Select the dataset CSV



Saving Train.csv to Train.csv
Saving Mapping.csv to Mapping.csv
Saving OutputFormat.csv to OutputFormat.csv
Saving Test.csv to Test.csv


In [ ]:
# Create a folder for preprocessing scripts
!mkdir -p /content/data_preprocessing


In [ ]:
with open('/content/data_preprocessing/text_preprocessing.py', 'w') as f:
    f.write("""import pandas as pd
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'@\w+', '', text)     # remove mentions
    text = re.sub(r'#\w+', '', text)     # remove hashtags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

def load_dataset(path):
    df = pd.read_csv(path)
    df['clean_text'] = df['text'].apply(clean_text)
    return df
""")


<>:7: SyntaxWarning: invalid escape sequence '\S'
<>:7: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-237935269.py:7: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub(r'http\S+', '', text)  # remove URLs


In [ ]:
with open('/content/data_preprocessing/text_preprocessing.py', 'w') as f:
    f.write("""<paste the code above>""")


In [ ]:
!ls /content/data_preprocessing


__pycache__  text_preprocessing.py


In [ ]:
from data_preprocessing.text_preprocessing import clean_text, load_dataset


## Preprocessing



In [ ]:
import pandas as pd

train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()


Train shape: (70000, 3)
Test shape: (25958, 3)


,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3


In [ ]:
# Check actual column names in train_df
print(train_df.columns.tolist())


['Unnamed: 0', 'TEXT', 'Label', 'clean_text']


In [ ]:
import pandas as pd
import re
import nltk

# Download required NLTK tokenizers
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

# Function to clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"@\w+", "", text)     # remove mentions
    text = re.sub(r"#\w+", "", text)     # remove hashtags
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    return text

# Strip any leading/trailing spaces from column names
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Apply cleaning
train_df['clean_text'] = train_df['TEXT'].astype(str).apply(clean_text)
test_df['clean_text'] = test_df['TEXT'].astype(str).apply(clean_text)

# Tokenize
from nltk.tokenize import word_tokenize
train_df['tokens'] = train_df['clean_text'].apply(word_tokenize)
test_df['tokens'] = test_df['clean_text'].apply(word_tokenize)

# Preview results
train_df.head()


,Unnamed: 0,TEXT,Label,clean_text,tokens
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0,vacation wasted port of\n,"[vacation, wasted, port, of]"
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1,oh wynwood youre so funny wynwood\n,"[oh, wynwood, youre, so, funny, wynwood]"
2,2,Been friends since 7th grade. Look at us now w...,2,been friends since 7th grade look at us now we...,"[been, friends, since, 7th, grade, look, at, u..."
3,3,This is what it looks like when someone loves ...,3,this is what it looks like when someone loves ...,"[this, is, what, it, looks, like, when, someon..."
4,4,RT @user this white family was invited to a Bl...,3,rt this white family was invited to a black b...,"[rt, this, white, family, was, invited, to, a,..."


In [ ]:
import os

# Create folder to save processed data
os.makedirs("/content/data/processed", exist_ok=True)

# Save processed train and test datasets
train_df.to_csv("/content/data/processed/train.csv", index=False)
test_df.to_csv("/content/data/processed/test.csv", index=False)


In [ ]:
!ls /content/data/processed


test.csv  train.csv


In [ ]:
import json

# Create metadata dictionary
metadata = {
    "train_shape": train_df.shape,
    "test_shape": test_df.shape,
    "train_columns": list(train_df.columns),
    "test_columns": list(test_df.columns)
}

# Save metadata as JSON
with open("/content/data/processed/metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)


In [ ]:
!cat /content/data/processed/metadata.json


{
    "train_shape": [
        70000,
        5
    ],
    "test_shape": [
        25958,
        5
    ],
    "train_columns": [
        "Unnamed: 0",
        "TEXT",
        "Label",
        "clean_text",
        "tokens"
    ],
    "test_columns": [
        "Unnamed: 0",
        "id",
        "TEXT",
        "clean_text",
        "tokens"
    ]
}

Baseline ML Model


In [2]:
# Clone your repo
!git clone https://github.com/nive62tech/emoji-translator.git

# Load datasets
import pandas as pd

train_df = pd.read_csv("emoji-translator/data/raw/Train.csv")
test_df = pd.read_csv("emoji-translator/data/raw/Test.csv")
mapping_df = pd.read_csv("emoji-translator/data/raw/Mapping.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print(train_df.head())


fatal: destination path 'emoji-translator' already exists and is not an empty directory.
Train shape: (70000, 3)
Test shape: (25958, 3)
   Unnamed: 0                                               TEXT  Label
0           0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1           1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2           2  Been friends since 7th grade. Look at us now w...      2
3           3  This is what it looks like when someone loves ...      3
4           4  RT @user this white family was invited to a Bl...      3


In [3]:
# 1. Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib


In [5]:
# 2. Load dataset
df = pd.read_csv("emoji-translator/data/raw/Train.csv")  # adjust path if needed

# 3. Split into features and labels
X = df['TEXT']   # text column
y = df['Label']  # label column

# 4. Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train size:", len(X_train))
print("Test size:", len(X_test))


Train size: 56000
Test size: 14000


In [6]:
# 5. TF-IDF + Logistic Regression pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # limit features for speed
    ('clf', LogisticRegression(max_iter=200))
])

# 6. Train model
pipeline.fit(X_train, y_train)


Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('clf', LogisticRegression(max_iter=200))])

In [7]:
# 7. Predictions & Evaluation
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.32571428571428573
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       282
           1       0.23      0.13      0.17       531
           2       0.26      0.23      0.24      1408
           3       0.34      0.51      0.41      1384
           4       0.12      0.01      0.02       372
           5       0.65      0.59      0.62       387
           6       0.14      0.04      0.06       431
           7       0.44      0.44      0.44       875
           8       0.14      0.01      0.02       377
           9       0.30      0.73      0.43      3049
          10       0.35      0.02      0.04       355
          11       0.69      0.44      0.53       509
          12       0.41      0.32      0.36       370
          13       0.30      0.15      0.20       644
          14       0.45      0.07      0.12       466
          15       0.24      0.04      0.06       728
          16       0.25      0.08      0.12       5

In [8]:
# 8. Save model
joblib.dump(pipeline, "baseline.pkl")


['baseline.pkl']

In [9]:
from google.colab import files
files.download('baseline.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Deep Learning Model

In [10]:
# 1. Install dependencies (if not already available in Colab)
!pip install tensorflow


In [11]:
# 2. Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout


In [16]:
from google.colab import files
uploaded = files.upload()  # Select Train.csv, Test.csv


Saving Mapping.csv to Mapping.csv
Saving Test.csv to Test.csv
Saving Train.csv to Train.csv


In [20]:
import pandas as pd

# Load the actual dataset
df = pd.read_csv("/content/emoji-translator/data/raw/Train.csv")  # correct path

# Split features and labels
X = df['TEXT']   # feature column
y = df['Label']  # target column

print("Dataset loaded successfully!")
print(df.head())


Dataset loaded successfully!
   Unnamed: 0                                               TEXT  Label
0           0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1           1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2           2  Been friends since 7th grade. Look at us now w...      2
3           3  This is what it looks like when someone loves ...      3
4           4  RT @user this white family was invited to a Bl...      3


In [21]:
# 4. Tokenization
max_words = 10000   # vocab size
max_len = 100       # max sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len, padding='post')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)


In [22]:
# 5. Build model
embedding_dim = 64

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=False),   # or GRU(64)
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(y.unique()), activation='softmax')  # multi-class classification
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# 6. Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=32
)


Epoch 1/5
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 110s 59ms/step - accuracy: 0.2062 - loss: 2.7790 - val_accuracy: 0.2178 - val_loss: 2.7343
Epoch 2/5
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 144s 60ms/step - accuracy: 0.2153 - loss: 2.7442 - val_accuracy: 0.2178 - val_loss: 2.7322
Epoch 3/5
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 109s 62ms/step - accuracy: 0.2194 - loss: 2.7341 - val_accuracy: 0.2178 - val_loss: 2.7318
Epoch 4/5
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 138s 60ms/step - accuracy: 0.2136 - loss: 2.7432 - val_accuracy: 0.2178 - val_loss: 2.7321
Epoch 5/5
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 102s 58ms/step - accuracy: 0.2144 - loss: 2.7398 - val_accuracy: 0.2178 - val_loss: 2.7333


In [24]:
# 7. Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


438/438 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.2194 - loss: 2.7307
Test Accuracy: 0.22


In [25]:
# 8. Save Keras model
model.save("emoji_model.h5")


In [26]:
from google.colab import files
files.download("emoji_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>